In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
import h5py
from sklearn.model_selection import train_test_split
import argparse

In [2]:
import copy
import logging
import torch
import torch.nn as nn
from torchvision import *
import random
import sys
import time
import cv2

In [3]:
img_size = 512

output_path = 'train.h5'
img_dir =  r'ISIC2018_Task1-2_Training_Input/'
mask_dir = 'ISIC2018_Task2_Training_GroundTruth_v3/'

IMG_FILE_REGEX = r'ISIC_(\d+).jpg'
MASK_FILE_REGEX = r'ISIC_(\d+)_attribute_(.*).png'

ISIC2019_FILE_REGEX = r'ISIC_(\d+).jpg'

ATTR_TO_INDEX = {
    'globules' : 0,
    'milia_like_cyst' : 1,
    'negative_network' : 2,
    'pigment_network' : 3,
    'streaks' : 4,
}

In [4]:
Back = '../../../../../594A/Dataset/ISIC challenge 2018 Task 1-2/'

img_dir =  Back + img_dir
mask_dir = Back + mask_dir

#  ISIC2018 train

In [5]:
df_ISIC2018 = pd.DataFrame(columns=['image', 'globules', 'milia_like_cyst', 
                                    'negative_network', 'pigment_network', 'streaks'])

In [6]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks


In [7]:
for img_file in os.listdir( img_dir ):
    if not img_file.endswith('.jpg'): continue
    img_id = str( re.search( IMG_FILE_REGEX, img_file ).group(1) )
    img_name = 'ISIC_' + img_id
    df_append = pd.DataFrame.from_records([{'image': img_name, 'globules': 2, 'milia_like_cyst': 2, 
                                      'negative_network': 2, 'pigment_network': 2, 'streaks': 2, 'pigment_network_pred': 2}])
    
    df_ISIC2018 = pd.concat([df_ISIC2018, df_append], ignore_index=True)

In [8]:
df_ISIC2018

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks,pigment_network_pred
0,ISIC_0008913,2,2,2,2,2,2.0
1,ISIC_0000418,2,2,2,2,2,2.0
2,ISIC_0015614,2,2,2,2,2,2.0
3,ISIC_0000079,2,2,2,2,2,2.0
4,ISIC_0013399,2,2,2,2,2,2.0
...,...,...,...,...,...,...,...
2589,ISIC_0010334,2,2,2,2,2,2.0
2590,ISIC_0010493,2,2,2,2,2,2.0
2591,ISIC_0012813,2,2,2,2,2,2.0
2592,ISIC_0011115,2,2,2,2,2,2.0


## 2. save binary input 

In [9]:
'''
for mask_file in os.listdir( mask_dir ):
    if not mask_file.endswith('.png'): continue
    mask = Image.open( os.path.join( mask_dir, mask_file ) )
    mask = mask.resize( (img_size, img_size) )
    mask = np.array( mask )
    
    assert mask.shape == (img_size, img_size)
    assert mask.max() <= 255
    
    # decide feature exist or not
    if mask.max() > 0:
        binary_label = 1
    else:
        binary_label = 0
    
    m = re.search(MASK_FILE_REGEX, mask_file)
    
    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]
    
    if m.group(2) ==  'globules':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label
    elif m.group(2) ==  'milia_like_cyst':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label
    elif m.group(2) ==  'negative_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = binary_label
    elif m.group(2) ==  'pigment_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('pigment_network')]  = binary_label
    elif m.group(2) ==  'streaks':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('streaks')]  = binary_label
'''

"\nfor mask_file in os.listdir( mask_dir ):\n    if not mask_file.endswith('.png'): continue\n    mask = Image.open( os.path.join( mask_dir, mask_file ) )\n    mask = mask.resize( (img_size, img_size) )\n    mask = np.array( mask )\n    \n    assert mask.shape == (img_size, img_size)\n    assert mask.max() <= 255\n    \n    # decide feature exist or not\n    if mask.max() > 0:\n        binary_label = 1\n    else:\n        binary_label = 0\n    \n    m = re.search(MASK_FILE_REGEX, mask_file)\n    \n    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]\n    \n    if m.group(2) ==  'globules':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label\n    elif m.group(2) ==  'milia_like_cyst':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label\n    elif m.group(2) ==  'negative_network':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = bina

In [10]:
#df_ISIC2018.to_csv('df_ISIC2018_val.csv', index=False)
csv_path  =  '../../../../../Explainable-NN/Data/train_fold_1.csv'
df_ISIC2018 = pd.read_csv(csv_path)

In [11]:
#df = df_ISIC2018.loc[(df_ISIC2018.pigment_network == 0) & (df_ISIC2018.globules == 0)
#                    & (df_ISIC2018.milia_like_cyst == 0) & (df_ISIC2018.negative_network == 1) & (df_ISIC2018.streaks == 0)]

# Train.h5py

In [ ]:
with h5py.File(output_path, "w") as f:
    count = len(df_ISIC2018)

    # store imgs to h5 file
    images = f.create_dataset( 'images', (count, 3, img_size, img_size), dtype=np.uint8 )
    image_ids = f.create_dataset( 'image_ids', (count,), dtype=int )
    img_id_to_h5idx = {}
    i = 0
    for idx in range(len(df_ISIC2018)):
        print( f'Processing image {i+1}/{count}' )
        img_file = df_ISIC2018.iloc[idx].image + '.jpg'
        img = Image.open( os.path.join( img_dir, img_file ) )
        img = img.resize( (img_size, img_size) )
        img = np.array( img )
        img = img.transpose(2, 0, 1)
        #(3,512,1024)
        assert img.shape == (3, img_size, img_size)
        assert img.max() <= 255
        images[i] = img
        img_id = int( re.search( IMG_FILE_REGEX, img_file ).group(1) )
        image_ids[i] = img_id
        img_id_to_h5idx[ img_id ] = i
        i += 1
        
    # store masks to h5 file
    i = 0
    masks = f.create_dataset('masks', (count, 5, img_size, img_size), dtype=np.uint8)
    attr_labels = f.create_dataset('labels', (count, 5), dtype=np.uint8)
    for mask_file in os.listdir( mask_dir ):
        if not mask_file.endswith('.png'): continue
        print( f'Processing mask {i+1}/{5*count}' )
        mask = Image.open( os.path.join( mask_dir, mask_file ) )
        mask = mask.resize( (img_size, img_size) )
        mask = np.array( mask )
        assert mask.shape == (img_size, img_size)
        assert mask.max() <= 255
        m = re.search(MASK_FILE_REGEX, mask_file)
        img_id, attr_id = int(m.group(1)), ATTR_TO_INDEX[ m.group(2) ]
        
        
        h5idx = img_id_to_h5idx[ img_id ]
        masks[ h5idx, attr_id ] = mask 
        if mask.max() > 0:
            attr_labels[h5idx, attr_id] = 1
        i += 1      

Processing image 1/2074
Processing image 2/2074
Processing image 3/2074
Processing image 4/2074
Processing image 5/2074
Processing image 6/2074
Processing image 7/2074
Processing image 8/2074
Processing image 9/2074
Processing image 10/2074
Processing image 11/2074
Processing image 12/2074
Processing image 13/2074
Processing image 14/2074
Processing image 15/2074
Processing image 16/2074
Processing image 17/2074
Processing image 18/2074
Processing image 19/2074
Processing image 20/2074
Processing image 21/2074
Processing image 22/2074
Processing image 23/2074
Processing image 24/2074
Processing image 25/2074
Processing image 26/2074
Processing image 27/2074
Processing image 28/2074
Processing image 29/2074
Processing image 30/2074
Processing image 31/2074
Processing image 32/2074
Processing image 33/2074
Processing image 34/2074
Processing image 35/2074
Processing image 36/2074
Processing image 37/2074
Processing image 38/2074
Processing image 39/2074
Processing image 40/2074
Processin

Processing image 321/2074
Processing image 322/2074
Processing image 323/2074
Processing image 324/2074
Processing image 325/2074
Processing image 326/2074
Processing image 327/2074
Processing image 328/2074
Processing image 329/2074
Processing image 330/2074
Processing image 331/2074
Processing image 332/2074
Processing image 333/2074
Processing image 334/2074
Processing image 335/2074
Processing image 336/2074
Processing image 337/2074
Processing image 338/2074
Processing image 339/2074
Processing image 340/2074
Processing image 341/2074
Processing image 342/2074
Processing image 343/2074
Processing image 344/2074
Processing image 345/2074
Processing image 346/2074
Processing image 347/2074
Processing image 348/2074
Processing image 349/2074
Processing image 350/2074
Processing image 351/2074
Processing image 352/2074
Processing image 353/2074
Processing image 354/2074
Processing image 355/2074
Processing image 356/2074
Processing image 357/2074
Processing image 358/2074
Processing i

Processing image 639/2074
Processing image 640/2074
Processing image 641/2074
Processing image 642/2074
Processing image 643/2074
Processing image 644/2074
Processing image 645/2074
Processing image 646/2074
Processing image 647/2074
Processing image 648/2074
Processing image 649/2074
Processing image 650/2074
Processing image 651/2074
Processing image 652/2074
Processing image 653/2074
Processing image 654/2074
Processing image 655/2074
Processing image 656/2074
Processing image 657/2074
Processing image 658/2074
Processing image 659/2074
Processing image 660/2074
Processing image 661/2074
Processing image 662/2074
Processing image 663/2074
Processing image 664/2074
Processing image 665/2074
Processing image 666/2074
Processing image 667/2074
Processing image 668/2074
Processing image 669/2074
Processing image 670/2074
Processing image 671/2074
Processing image 672/2074
Processing image 673/2074
Processing image 674/2074
Processing image 675/2074
Processing image 676/2074
Processing i

Processing image 955/2074
Processing image 956/2074
Processing image 957/2074
Processing image 958/2074
Processing image 959/2074
Processing image 960/2074
Processing image 961/2074
Processing image 962/2074
Processing image 963/2074
Processing image 964/2074
Processing image 965/2074
Processing image 966/2074
Processing image 967/2074
Processing image 968/2074
Processing image 969/2074
Processing image 970/2074
Processing image 971/2074
Processing image 972/2074
Processing image 973/2074
Processing image 974/2074
Processing image 975/2074
Processing image 976/2074
Processing image 977/2074
Processing image 978/2074
Processing image 979/2074
Processing image 980/2074
Processing image 981/2074
Processing image 982/2074
Processing image 983/2074
Processing image 984/2074
Processing image 985/2074
Processing image 986/2074
Processing image 987/2074
Processing image 988/2074
Processing image 989/2074
Processing image 990/2074
Processing image 991/2074
Processing image 992/2074
Processing i

Processing image 1261/2074
Processing image 1262/2074
Processing image 1263/2074
Processing image 1264/2074
Processing image 1265/2074
Processing image 1266/2074
Processing image 1267/2074
Processing image 1268/2074
Processing image 1269/2074
Processing image 1270/2074
Processing image 1271/2074
Processing image 1272/2074
Processing image 1273/2074
Processing image 1274/2074
Processing image 1275/2074
Processing image 1276/2074
Processing image 1277/2074
Processing image 1278/2074
Processing image 1279/2074
Processing image 1280/2074
Processing image 1281/2074
Processing image 1282/2074
Processing image 1283/2074
Processing image 1284/2074
Processing image 1285/2074
Processing image 1286/2074
Processing image 1287/2074
Processing image 1288/2074
Processing image 1289/2074
Processing image 1290/2074
Processing image 1291/2074
Processing image 1292/2074
Processing image 1293/2074
Processing image 1294/2074
Processing image 1295/2074
Processing image 1296/2074
Processing image 1297/2074
P

Processing image 1565/2074
Processing image 1566/2074
Processing image 1567/2074
Processing image 1568/2074
Processing image 1569/2074
Processing image 1570/2074
Processing image 1571/2074
Processing image 1572/2074
Processing image 1573/2074
Processing image 1574/2074
Processing image 1575/2074
Processing image 1576/2074
Processing image 1577/2074
Processing image 1578/2074
Processing image 1579/2074
Processing image 1580/2074
Processing image 1581/2074
Processing image 1582/2074
Processing image 1583/2074
Processing image 1584/2074
Processing image 1585/2074
Processing image 1586/2074
Processing image 1587/2074
Processing image 1588/2074
Processing image 1589/2074
Processing image 1590/2074
Processing image 1591/2074
Processing image 1592/2074
Processing image 1593/2074
Processing image 1594/2074
Processing image 1595/2074
Processing image 1596/2074
Processing image 1597/2074
Processing image 1598/2074
Processing image 1599/2074
Processing image 1600/2074
Processing image 1601/2074
P